In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from scipy.optimize import curve_fit
from scipy.optimize import linear_sum_assignment
import tqdm



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [84]:
LS133 = "./ntupleStubsBmtfGmt_LS133.root"
gmt_awk_LS133 = OpenRootFile(LS133, gmt_features)
bmtf_awk_LS133 = OpenRootFile(LS133, bmtf_features)

In [85]:
Gmt_Phi_LS133 = gmt_awk_LS133.gmtPhi
Bmtf_Phi_LS133 = bmtf_awk_LS133.bmtfPhi

Gmt_Eta_LS133 = gmt_awk_LS133.gmtEta
Bmtf_Eta_LS133 = bmtf_awk_LS133.bmtfEta

nGmtMuons_LS133 = gmt_awk_LS133.nGmtMuons
nBmtfMuons_LS133 = bmtf_awk_LS133.nBmtfMuons

In [92]:
def Delta(BMTF, GMT):
  return BMTF - GMT

def DeltaR(BmtfPhi, GmtPhi, BmtfEta, GmtEta):
  return np.sqrt((BmtfPhi - GmtPhi)**2 + (BmtfEta - GmtEta)**2)

In [107]:
MuonsProperties = []

use_Bmtf = True


for i in tqdm.tqdm(range(50000)):

  DeltaRDistBMTF = []
  IndexMinBMTF = []
  DeltaRDistGMT = []
  IndexMinGMT = []

  if nGmtMuons_LS133[i] == 0 and nBmtfMuons_LS133[i] == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = []
  
  elif nBmtfMuons_LS133[i] == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = [range(nGmtMuons_LS133[i])]

  elif nGmtMuons_LS133[i] == 0:
    unmatched_Bmtf = [range(nBmtfMuons_LS133[i])]
    unmatched_Gmt = []

  else:
    #Primo metodo 
    for j in range(nBmtfMuons_LS133[i]):

      DeltaRTemp = DeltaR(Bmtf_Phi_LS133[i, j], Gmt_Phi_LS133[i, :], Bmtf_Eta_LS133[i, j], Gmt_Eta_LS133[i, :])

      minDeltaR = np.min(DeltaRTemp)
  
      indexTempBMTF = np.argmin(DeltaRTemp)
      DeltaRDistBMTF.append(minDeltaR)

      #Tolgo il valore che corrisponde al più basso deltaR in modo che non venga ripetuto




      if minDeltaR < 0.4:
        DeltaPhi = Delta(Bmtf_Phi_LS133[i, j], Gmt_Phi_LS133[i, indexTempBMTF])
        DeltaEta = Delta(Bmtf_Eta_LS133[i, j], Gmt_Eta_LS133[i, indexTempBMTF])

      else:
        #Se DeltaR > 0.4 (primo metodo) non c'è match con i muoni del GMT
        unmatched_Gmt = [range(nGmtMuons_LS133[i])]

    
    #Secondo metodo 
    for k in range(nGmtMuons_LS133[i]):
      DeltaRTemp = DeltaR(Bmtf_Phi_LS133[i, :], Gmt_Phi_LS133[i, k], Bmtf_Eta_LS133[i, :], Gmt_Eta_LS133[i, k])

      minDeltaR = np.min(DeltaRTemp)
  
      indexTempGMT = np.argmin(DeltaRTemp)
      DeltaRDistGMT.append(minDeltaR)


      #Tolgo il valore

      if minDeltaR < 0.4:
        DeltaPhi = Delta(Bmtf_Phi_LS133[i, indexTempGMT], Gmt_Phi_LS133[i, k])
        DeltaEta = Delta(Bmtf_Eta_LS133[i, indexTempGMT], Gmt_Eta_LS133[i, k])

      else:
        unmatched_Bmtf = [range(nBmtfMuons_LS133[i])]

   

  Events = {
    'unmatched_Gmt_Events' : unmatched_Gmt,
    'unmatched_Bmtf_Events': unmatched_Bmtf, 
    'DeltaR': DeltaRDistBMTF,
  }

  MuonsProperties.append(Events)


  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [00:46<00:00, 1077.62it/s]


In [108]:
MuonsProperties = ak.Array(MuonsProperties)
print(ak.sum(ak.num(MuonsProperties.unmatched_Bmtf_Events)))
print(ak.sum(ak.num(MuonsProperties.unmatched_Gmt_Events)))

2
32
